# 1. Exploration

In [1]:
import json
from queue import PriorityQueue
f1 = open('../data/G.json')
f2 = open('../data/Coord.json')
f3 = open('../data/Dist.json')
f4 = open('../data/Cost.json')
MAX_BUDGET = 287932

In [2]:
from collections import defaultdict
G = json.load(f1)
# h(n)
Coord = json.load(f2)
# g(n)
Dist = json.load(f3)
Cost = defaultdict(int, json.load(f4))

In [3]:
def UCS(graph, source, target):
    visited = set()
    # priority queue of score, path. Lower scores are higher on priority.
    queue = PriorityQueue()
    queue.put((0, [source], 0))
    while not queue.empty():
        # get the highest priority path
        candidate = queue.get()
        cur_node = candidate[1][-1]
        visited.add(cur_node)
        if cur_node == target:
            return candidate
        # add all the edges to the priority queue
        for node in graph[cur_node]:
            if node in visited:
                continue
            # create a new path with the node from the edge
            new_path = list(candidate[1]) + [node]
            # add distance and new path to queue.
            queue.put((candidate[0] + Dist[f'{cur_node},{node}'], 
                       new_path, 
                       candidate[2]+Cost[f'{cur_node},{node}']))
    return None, None, None

In [4]:
distance, path, cost = UCS(G, '1', '50')

In [5]:
print('Shortest path:', '->'.join(node for node in path) + '.')
print(f'Shortest distance: {distance}.')
print(f'Total energy cost: {cost}')

Shortest path: 1->1363->1358->1357->1356->1276->1273->1277->1269->1267->1268->1284->1283->1282->1255->1253->1260->1259->1249->1246->963->964->962->1002->952->1000->998->994->995->996->987->986->979->980->969->977->989->990->991->2369->2366->2340->2338->2339->2333->2334->2329->2029->2027->2019->2022->2000->1996->1997->1993->1992->1989->1984->2001->1900->1875->1874->1965->1963->1964->1923->1944->1945->1938->1937->1939->1935->1931->1934->1673->1675->1674->1837->1671->1828->1825->1817->1815->1634->1814->1813->1632->1631->1742->1741->1740->1739->1591->1689->1585->1584->1688->1579->1679->1677->104->5680->5418->5431->5425->5424->5422->5413->5412->5411->66->5392->5391->5388->5291->5278->5289->5290->5283->5284->5280->50.
Shortest distance: 148648.63722140007.
Total energy cost: 294853


In [6]:
def get_graph(fp='../data/G.json'):
    f = open(fp)
    return json.load(f)

In [7]:
def path_dist(path):
    total_dist = 0
    for i in range(1, len(path)):
        total_dist += Dist[f'{path[i-1]},{path[i]}']
    return total_dist

In [8]:
def path_cost(path):
    total_cost = 0
    for i in range(1, len(path)):
        total_cost += Cost[f'{path[i-1]},{path[i]}']
    return total_cost

In [9]:
def yens_ksp(graph, func, source, target, budget, K=5, A=None):
    if not A:
        A = [func(graph, source, target)[1]]
    B = PriorityQueue() #Queue of candidate paths

    for k in range(1, K):
        for i in range(len(A[k-1]) - 1):
            G = get_graph() #New graph for each iteration
            spur_node = A[k-1][i]
            root_path = A[k-1][:i]
            for path in A:
                if len(path) - 1 > i and root_path == path[:i]:
                    # remove edge connecting root path to spur node from Graph, if edge isn't
                    # already deleted
                    if path[i+1] in G[path[i]]:
                        G[path[i]].remove(path[i+1])
            _, spur_path, _ = func(G, spur_node, target)
            # if there exists a path from spur node to target
            if spur_path:
                found_in_B = False
                for paths in B.queue:
                    # if path exists then dont add total_path info in.
                    if paths[1]==spur_path:
                        found_in_B = True
                        break
                # path is new to B, add it in
                if not found_in_B:
                    total_path = root_path + spur_path
                    B.put((path_dist(total_path), total_path, path_cost(total_path)))
        # since B is already sorted, we first try to find whether shortest paths in B
        # satisfy our budget constraint. If not, find the shortest path and add to A.
        while B:
            dist, path, cost = B.get()
            # this is a k-th version of shortest path
            if path not in A:
                if cost <= budget:
                    return path, dist, cost
                A.append(path)
    return A, None, None

In [10]:
path2, _, _ = yens_ksp(G, UCS, '1', '50', MAX_BUDGET, K=1000)

1


In [12]:
print('Shortest path2:', '->'.join(node for node in path2) + '.')
print(f'Shortest distance: {path_dist(path2)}.')
print(f'Total energy cost: {path_cost(path2)}.')

Shortest path2: 1->1363->1358->1357->1356->1276->1273->1277->1269->1267->1268->1284->1283->1282->1255->1253->1260->1259->1249->1246->963->964->962->1002->952->1000->998->994->995->996->987->986->979->980->969->977->989->990->991->2369->2366->2340->2338->2339->2333->2334->2329->2029->2027->2019->2022->2000->1996->1997->1993->1992->1989->1984->2001->1900->1875->1874->1965->1963->1964->1923->1944->1945->1938->1937->1939->1935->1931->1934->1673->1675->1674->1837->1671->1828->1825->1817->1815->1634->1814->1813->1632->1631->1742->1741->1740->1739->1591->1689->1585->1584->1688->1579->1679->1677->104->5680->5418->5431->5425->5429->5426->5428->5434->5435->5433->5436->5398->5404->5402->5396->5395->5292->5282->5283->5284->5280->50.
Shortest distance: 150784.60722193593.
Total energy cost: 287931.
